In [1]:
import scipy.io
import numpy as np
import tensorflow as tf
import os
import logging
import datetime
import shutil
import argparse
from collections import defaultdict
import random
from collections import Counter
os.environ["CUDA_VISIBLE_DEVICES"] = "1"

## Data preprocessing
### Data partitioning

In [2]:
np.random.seed(7)
random.seed(7)

In [3]:
ratio = 0.6 # train:val:test = ratio:(1-ratio)/2:(1-ratio)/2

In [4]:
mat = scipy.io.loadmat('douban.mat')
UM, UU, UG, UL, MD, MA, MT = (x.tocoo() for x in list(mat['relation'][0]))

In [5]:
user_biz = defaultdict(list)
for u, b, d in zip(UM.row, UM.col, UM.data):
    if d >= 5:
        user_biz[u].append(b)

user_biz_train = defaultdict(list)
user_biz_val = defaultdict(list)
user_biz_test = defaultdict(list)
train_set = set()
val_set = set()
test_set = set()
for u in user_biz:
    if len(user_biz[u]) >= 5:
        val = int(len(user_biz[u]) * ratio)
        test = int(len(user_biz[u]) * (ratio+(1-ratio)/2))
        random.shuffle(user_biz[u])
        user_biz_train[u] = user_biz[u][:val]
        for b in user_biz[u][:val]:
            train_set.add((u, b))
        user_biz_val[u] = user_biz[u][val:test]
        for b in user_biz[u][val:test]:
            val_set.add((u, b))
        user_biz_test[u] = user_biz[u][test:]
        for b in user_biz[u][test:]:
            test_set.add((u, b))

In [6]:
train_mask = []
val_mask = []
test_mask = []
for ind, ub in enumerate(zip(UM.row, UM.col)):
    if ub in train_set:
        train_mask.append(ind)
    if ub in val_set:
        val_mask.append(ind)
    if ub in test_set:
        test_mask.append(ind)
train_mask = np.array(train_mask)
val_mask = np.array(val_mask)
test_mask = np.array(test_mask)

In [7]:
user_biz_train = np.zeros([UM.shape[0], UM.shape[1]])
for u, b in zip(UM.row[train_mask], UM.col[train_mask]):
    user_biz_train[u, b] += 1.

In [8]:
sum_tmp = np.sum(user_biz_train, -1)
user_non_zero_ind = np.array([i for i in range(UM.shape[0]) if sum_tmp[i] > 0])
user_non_zero_set = set(user_non_zero_ind)

sum_tmp = np.sum(user_biz_train[user_non_zero_ind, :], 0)
biz_non_zero_mask = (sum_tmp != 0).astype(float)
biz_non_zero_set = set([i for i in range(UM.shape[1])])

In [9]:
user_biz_val = np.zeros([UM.shape[0], UM.shape[1]])
user_biz_val_dict = defaultdict(list)
for u, b in zip(UM.row[val_mask], UM.col[val_mask]):
    if u in user_non_zero_set and b in biz_non_zero_set:
        user_biz_val[u, b] += 1.
    user_biz_val_dict[u].append(b)

user_biz_test = np.zeros([UM.shape[0], UM.shape[1]])
user_biz_test_dict = defaultdict(list)
for u, b in zip(UM.row[test_mask], UM.col[test_mask]):
    if u in user_non_zero_set and b in biz_non_zero_set:
        user_biz_test[u, b] += 1.
    user_biz_test_dict[u].append(b)

In [10]:
# data structure needed for the convenience of evaluation
val_set_u = list(user_non_zero_set)
val_set_mask = []
val_set_set = []
for user in val_set_u:
    val_set_mask.append(biz_non_zero_mask - user_biz_train[user, :] - user_biz_test[user, :])
    val_set_set.append(set(user_biz_val_dict[user]))

test_set_u = list(user_non_zero_set)
test_set_mask = []
test_set_set = []
for user in test_set_u:
    test_set_mask.append(biz_non_zero_mask - user_biz_train[user, :] - user_biz_val[user, :])
    test_set_set.append(set(user_biz_test_dict[user]))

In [11]:
# normalize the training data
user_biz_train = user_biz_train[user_non_zero_ind, :]
user_biz_train = user_biz_train/np.sum(user_biz_train, -1).reshape([-1, 1])

## Model
### HINs & parameters

In [12]:
# sparse tensors for adjacency matrices

UM_train = tf.SparseTensor(indices=np.array([UM.row[train_mask], UM.col[train_mask]]).transpose(),
                           values=np.ones(len(train_mask)).astype(np.float32),
                           dense_shape=UM.shape)
UU_t = tf.SparseTensor(indices=np.array([UU.row, UU.col]).transpose(), values=UU.data.astype(np.float32), dense_shape=UU.shape)
UG_t = tf.SparseTensor(indices=np.array([UG.row, UG.col]).transpose(), values=UG.data.astype(np.float32), dense_shape=UG.shape)
UL_t = tf.SparseTensor(indices=np.array([UL.row, UL.col]).transpose(), values=UL.data.astype(np.float32), dense_shape=UL.shape)
MD_t = tf.SparseTensor(indices=np.array([MD.row, MD.col]).transpose(), values=MD.data.astype(np.float32), dense_shape=MD.shape)
MA_t = tf.SparseTensor(indices=np.array([MA.row, MA.col]).transpose(), values=MA.data.astype(np.float32), dense_shape=MA.shape)
MT_t = tf.SparseTensor(indices=np.array([MT.row, MT.col]).transpose(), values=MT.data.astype(np.float32), dense_shape=MT.shape)

In [13]:
# paras

num_U = UU.shape[0]
num_G = UG.shape[1]
num_L = UL.shape[1]
num_M = MD.shape[0]
num_A = MA.shape[1]
num_D = MD.shape[1]
num_T = MT.shape[1]

dim = 64
batch_size = 64

In [14]:
# embeddings
U_embeddings = tf.get_variable("U_embeddings", [num_U, dim], initializer=tf.truncated_normal_initializer(stddev=0.01), regularizer=tf.contrib.layers.l2_regularizer(scale=0.1))
M_embeddings = tf.get_variable("B_embeddings", [num_M, dim], initializer=tf.truncated_normal_initializer(stddev=0.01), regularizer=tf.contrib.layers.l2_regularizer(scale=0.1))
G_embeddings = tf.get_variable("G_embeddings", [num_G, dim], initializer=tf.truncated_normal_initializer(stddev=0.01), regularizer=tf.contrib.layers.l2_regularizer(scale=0.1))
L_embeddings = tf.get_variable("L_embeddings", [num_L, dim], initializer=tf.truncated_normal_initializer(stddev=0.01), regularizer=tf.contrib.layers.l2_regularizer(scale=0.1))
A_embeddings = tf.get_variable("A_embeddings", [num_A, dim], initializer=tf.truncated_normal_initializer(stddev=0.01), regularizer=tf.contrib.layers.l2_regularizer(scale=0.1))
D_embeddings = tf.get_variable("D_embeddings", [num_D, dim], initializer=tf.truncated_normal_initializer(stddev=0.01), regularizer=tf.contrib.layers.l2_regularizer(scale=0.1))
T_embeddings = tf.get_variable("T_embeddings", [num_T, dim], initializer=tf.truncated_normal_initializer(stddev=0.01), regularizer=tf.contrib.layers.l2_regularizer(scale=0.1))

In [15]:
# user & item lookup table
U_vec = U_embeddings + tf.sparse_tensor_dense_matmul(tf.sparse_softmax(UM_train), M_embeddings) + \
        tf.sparse_tensor_dense_matmul(tf.sparse_softmax(UG_t), G_embeddings) + \
        tf.sparse_tensor_dense_matmul(tf.sparse_softmax(UU_t), U_embeddings) + \
        tf.sparse_tensor_dense_matmul(tf.sparse_softmax(UL_t), L_embeddings)
U_vec = tf.nn.tanh(U_vec)

M_vec = M_embeddings + \
        tf.sparse_tensor_dense_matmul(tf.sparse_softmax(MA_t), A_embeddings) + \
        tf.sparse_tensor_dense_matmul(tf.sparse_softmax(MD_t), D_embeddings) + \
        tf.sparse_tensor_dense_matmul(tf.sparse_softmax(MT_t), T_embeddings)
M_vec = tf.nn.tanh(M_vec)

In [16]:
# placeholders
ux = tf.placeholder(tf.int32, shape=(None,))
uy = tf.placeholder(tf.float32, shape=(None, num_M))

### Embedding-based model

In [17]:
uvec = tf.tile(tf.expand_dims(tf.nn.embedding_lookup(U_vec, ux), 1), [1, num_M, 1])
bvec = tf.tile(tf.expand_dims(tf.nn.embedding_lookup(M_vec, tf.range(0, num_M)), 0), [tf.shape(uvec)[0], 1, 1])

In [18]:
x = tf.concat([uvec * bvec], axis=-1)
y_emb_logit = tf.squeeze(tf.layers.dense(x, 1, name='output_2', kernel_regularizer=tf.contrib.layers.l2_regularizer(scale=0.1)))
y_inference = tf.nn.softmax(y_emb_logit, -1)

loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=y_emb_logit, labels=uy))

Instructions for updating:

Future major versions of TensorFlow will allow gradients to flow
into the labels input on backprop by default.

See `tf.nn.softmax_cross_entropy_with_logits_v2`.



### Path-based model

In [19]:
# learnable adjacency matrices

UM_e = tf.Variable(np.zeros(shape=len(train_mask)), trainable=True, dtype=tf.float32)
UU_e = tf.Variable(np.zeros(shape=len(UU.row)), trainable=True, dtype=tf.float32)
UG_e = tf.Variable(np.zeros(shape=len(UG.row)), trainable=True, dtype=tf.float32)
UL_e = tf.Variable(np.zeros(shape=len(UL.row)), trainable=True, dtype=tf.float32)
MD_e = tf.Variable(np.zeros(shape=len(MD.row)), trainable=True, dtype=tf.float32)
MA_e = tf.Variable(np.zeros(shape=len(MA.row)), trainable=True, dtype=tf.float32)
MT_e = tf.Variable(np.zeros(shape=len(MT.row)), trainable=True, dtype=tf.float32)
M_p = tf.Variable(np.zeros(shape=num_M), trainable=True, dtype=tf.float32)

UM_t = tf.SparseTensor(indices=np.array([UM.row[train_mask], UM.col[train_mask]]).transpose(),
                       values=tf.nn.softplus(UM_e),
                       dense_shape=UM.shape)
UU_t = tf.SparseTensor(indices=np.array([UU.row, UU.col]).transpose(),
                       values=tf.nn.softplus(UU_e),
                       dense_shape=UU.shape)
UG_t = tf.SparseTensor(indices=np.array([UG.row, UG.col]).transpose(),
                       values=tf.nn.softplus(UG_e),
                       dense_shape=UG.shape)
UL_t = tf.SparseTensor(indices=np.array([UL.row, UL.col]).transpose(),
                       values=tf.nn.softplus(UL_e),
                       dense_shape=UL.shape)
MD_t = tf.SparseTensor(indices=np.array([MD.row, MD.col]).transpose(),
                       values=tf.nn.softplus(MD_e),
                       dense_shape=MD.shape)
MA_t = tf.SparseTensor(indices=np.array([MA.row, MA.col]).transpose(),
                       values=tf.nn.softplus(MA_e),
                       dense_shape=MA.shape)
MT_t = tf.SparseTensor(indices=np.array([MT.row, MT.col]).transpose(),
                       values=tf.nn.softplus(MT_e),
                       dense_shape=MT.shape)

In [20]:
u_one_hot = tf.one_hot(ux, depth=num_U)
# meta-paths

# P1: UMUM
M_1 = tf.transpose(tf.sparse_tensor_dense_matmul(UM_t, u_one_hot, adjoint_a=True, adjoint_b=True)) + 1e-10
M_1 = M_1 / tf.reshape(tf.reduce_sum(M_1, axis=1), [-1, 1])
U_1 = tf.transpose(tf.sparse_tensor_dense_matmul(UM_t, M_1, adjoint_b=True)) + 1e-10
U_1 = U_1 / tf.reshape(tf.reduce_sum(U_1, axis=1), [-1, 1])
M_2 = tf.transpose(tf.sparse_tensor_dense_matmul(UM_t, U_1, adjoint_a=True, adjoint_b=True)) + 1e-10
M_2 = M_2 / tf.reshape(tf.reduce_sum(M_2, axis=1), [-1, 1])

# P2: UMUMUM
U_4 = tf.transpose(tf.sparse_tensor_dense_matmul(UM_t, M_2, adjoint_b=True)) + 1e-10
U_4 = U_4 / tf.reshape(tf.reduce_sum(U_4, axis=1), [-1, 1])
M_5 = tf.transpose(tf.sparse_tensor_dense_matmul(UM_t, U_4, adjoint_a=True, adjoint_b=True)) + 1e-10
M_5 = M_5 / tf.reshape(tf.reduce_sum(M_5, axis=1), [-1, 1])

# P3: UUM
U_2 = tf.transpose(tf.sparse_tensor_dense_matmul(UU_t, u_one_hot, adjoint_b=True)) + 1e-10
U_2 = U_2 / tf.reshape(tf.reduce_sum(U_2, axis=1), [-1, 1])
M_3 = tf.transpose(tf.sparse_tensor_dense_matmul(UM_t, U_2, adjoint_a=True, adjoint_b=True)) + 1e-10
M_3 = M_3 / tf.reshape(tf.reduce_sum(M_3, axis=1), [-1, 1])

# P4: UGrUM
G_1 = tf.transpose(tf.sparse_tensor_dense_matmul(UG_t, u_one_hot, adjoint_a=True, adjoint_b=True)) + 1e-10
G_1 = G_1 / tf.reshape(tf.reduce_sum(G_1, axis=1), [-1, 1])
U_3 = tf.transpose(tf.sparse_tensor_dense_matmul(UG_t, G_1, adjoint_b=True)) + 1e-10
U_3 = U_3 / tf.reshape(tf.reduce_sum(U_3, axis=1), [-1, 1])
M_9 = tf.transpose(tf.sparse_tensor_dense_matmul(UM_t, U_3, adjoint_a=True, adjoint_b=True)) + 1e-10
M_9 = M_9 / tf.reshape(tf.reduce_sum(M_9, axis=1), [-1, 1])

# P5: ULUM
L_1 = tf.transpose(tf.sparse_tensor_dense_matmul(UL_t, u_one_hot, adjoint_a=True, adjoint_b=True)) + 1e-10
L_1 = L_1 / tf.reshape(tf.reduce_sum(L_1, axis=1), [-1, 1])
U_4 = tf.transpose(tf.sparse_tensor_dense_matmul(UL_t, L_1, adjoint_b=True)) + 1e-10
U_4 = U_4 / tf.reshape(tf.reduce_sum(U_4, axis=1), [-1, 1])
M_10 = tf.transpose(tf.sparse_tensor_dense_matmul(UM_t, U_4, adjoint_a=True, adjoint_b=True)) + 1e-10
M_10 = M_10 / tf.reshape(tf.reduce_sum(M_10, axis=1), [-1, 1])

# p6: UMDM
D_1 = tf.transpose(tf.sparse_tensor_dense_matmul(MD_t, M_1, adjoint_a=True, adjoint_b=True)) + 1e-10
D_1 = D_1 / tf.reshape(tf.reduce_sum(D_1, axis=1), [-1, 1])
M_8 = tf.transpose(tf.sparse_tensor_dense_matmul(MD_t, D_1, adjoint_b=True)) + 1e-10
M_8 = M_8 / tf.reshape(tf.reduce_sum(M_8, axis=1), [-1, 1])

# P7: UMAM
A_1 = tf.transpose(tf.sparse_tensor_dense_matmul(MA_t, M_1, adjoint_a=True, adjoint_b=True)) + 1e-10
A_1 = A_1 / tf.reshape(tf.reduce_sum(A_1, axis=1), [-1, 1])
M_7 = tf.transpose(tf.sparse_tensor_dense_matmul(MA_t, A_1, adjoint_b=True)) + 1e-10
M_7 = M_7 / tf.reshape(tf.reduce_sum(M_7, axis=1), [-1, 1])

# P8: UMGeM
T_1 = tf.transpose(tf.sparse_tensor_dense_matmul(MT_t, M_1, adjoint_a=True, adjoint_b=True)) + 1e-10
T_1 = T_1 / tf.reshape(tf.reduce_sum(T_1, axis=1), [-1, 1])
M_6 = tf.transpose(tf.sparse_tensor_dense_matmul(MT_t, T_1, adjoint_b=True)) + 1e-10
M_6 = M_6 / tf.reshape(tf.reduce_sum(M_6, axis=1), [-1, 1])

### Loss function

In [21]:
# L1
r = tf.nn.softmax(tf.Variable(np.ones(shape=[8]), dtype=tf.float32, trainable=True))
y_path = tf.einsum('i,ijk->jk', r, tf.stack([M_2, M_5, M_3, M_9, M_10, M_8, M_7, M_6], axis=0))
# L2
loss_path = tf.reduce_mean(tf.reduce_sum(- uy * tf.log(y_path), -1))

# L3
kl_div = tf.reduce_mean(tf.reduce_sum(- tf.log(tf.nn.softmax(y_emb_logit, -1)) * y_path + y_path * tf.log(y_path), -1))

reg = tf.get_collection(tf.GraphKeys.REGULARIZATION_LOSSES)

In [22]:
# optimizer
loss_beta = tf.placeholder(tf.float32, shape=(), name='loss_beta')
opt = tf.train.AdamOptimizer().minimize(loss_beta * loss + kl_div + 0.2 * loss_path + 1e-6 * tf.reduce_sum(reg))

opt_base = tf.train.AdamOptimizer().minimize(loss + 1e-6 * tf.reduce_sum(reg))

## Training & Evaluation

In [23]:
def evaluate(u_batch, mask_batch, set_batch, y_val, cutoff=[20, 20, 100]):
    hit = []
    recall = []
    ndcg = []
    y_val_argsort = np.argsort(-y_val, axis=-1)[:, :cutoff[2]]
    for i in range(len(u_batch)):
        has_hit = 0
        recall_ = 0.
        dcg_max = 0.
        dcg = 0.
        top_k = y_val_argsort[i]
        h = set_batch[i]
        for ind, b_rec in enumerate(top_k):
            if ind < len(h):
                dcg_max += 1. / np.log2(ind + 2)
            if b_rec in h:
                if ind < cutoff[0]:
                    has_hit = 1
                if ind < cutoff[1]:
                    recall_ += 1.
                dcg += 1. / np.log2(ind + 2)
        
        hit.append(has_hit)
        ndcg.append(dcg / dcg_max)
        recall_ /= min(len(h), cutoff[1])
        recall.append(recall_)
    return hit, recall, ndcg

In [24]:
config = tf.ConfigProto()
config.gpu_options.allow_growth = True
sess = tf.Session(config=config)
sess.run(tf.global_variables_initializer())

train_loss = 0.
train_auc = 0.
train_hit = 0.
loss_cnt = 0.
best_ndcg = 0.
counter = 0
base = False  # whether to train the base model alone 
saver = tf.train.Saver(max_to_keep=None)


# sess.graph.finalize()
for epoch in range(100):
    train_loss = 0.
    train_kl = 0.
    loss_cnt = 0.

    for i in range(0, user_biz_train.shape[0], batch_size):
        uy_batch = user_biz_train[i:i+batch_size, :]
        ux_batch = user_non_zero_ind[i:i+batch_size]
        if base:
            _, loss_val, kl_val = sess.run([opt_base, loss, kl_div], feed_dict={ux: ux_batch, uy: uy_batch})
        else:
            _, loss_val, kl_val = sess.run([opt, loss, kl_div], feed_dict={ux: ux_batch, uy: uy_batch, loss_beta: max(2 * (0.9 ** epoch), 0.2)})

        train_loss += loss_val
        train_kl += kl_val
        loss_cnt += 1

    print("{}, {}, {}".format(epoch, train_loss/loss_cnt, train_kl/loss_cnt))

    val_hit = []
    val_recall = []
    val_ndcg = []
    for i in range(0, len(val_set_u), batch_size):
        u_batch = val_set_u[i:i+batch_size]
        mask_batch = val_set_mask[i:i+batch_size]
        set_batch = val_set_set[i:i+batch_size]
        y_val = sess.run(y_inference, feed_dict={ux: u_batch})
        y_val *= np.array(mask_batch)
        hit_, recall_, ndcg_ = evaluate(u_batch, mask_batch, set_batch, y_val, cutoff=[5, 20, 100])
        val_hit += hit_
        val_recall += recall_
        val_ndcg += ndcg_
    val_hit = np.array(val_hit)
    val_recall = np.array(val_recall)
    val_ndcg = np.array(val_ndcg)
    print("{}, {}, {}".format(val_hit.mean(), val_recall.mean(), val_ndcg.mean()))
    
    if val_ndcg.mean() >= best_ndcg:
        best_ndcg = val_ndcg.mean()
        counter = 0
        saver.save(sess, './saved_model_%s.bin'%('base' if base else 'proposed'))

    counter += 1
    if counter > 3:
        break

saver.restore(sess, './saved_model_%s.bin'%('base' if base else 'proposed'))
test_hit = []
test_recall = []
test_ndcg = []
for i in range(0, len(test_set_u), batch_size):
    u_batch = test_set_u[i:i+batch_size]
    mask_batch = test_set_mask[i:i+batch_size]
    set_batch = test_set_set[i:i+batch_size]
    y_val = sess.run(y_inference, feed_dict={ux: u_batch})
    y_val *= np.array(mask_batch)
    hit_, recall_, ndcg_ = evaluate(u_batch, mask_batch, set_batch, y_val, cutoff=[5, 20, 100])
    test_hit += hit_
    test_recall += recall_
    test_ndcg += ndcg_
test_hit = np.array(test_hit)
test_recall = np.array(test_recall)
test_ndcg = np.array(test_ndcg)
print("{}, {}, {}".format(test_hit.mean(), test_recall.mean(), test_ndcg.mean()))

0, 9.00611017846, 1.35025826983
0.200338123415, 0.116954880505, 0.143510762807
INFO:tensorflow:./saved_model_proposed.bin is not in all_model_checkpoint_paths. Manually adding it.
1, 7.67179508037, 0.749819595535
0.348126232742, 0.196032418228, 0.208448938664
INFO:tensorflow:./saved_model_proposed.bin is not in all_model_checkpoint_paths. Manually adding it.
2, 7.3971736023, 0.586482166707
0.35235277543, 0.194461192373, 0.20839312237
3, 7.28640626357, 0.479596162702
0.351507466892, 0.194288567614, 0.208869640789
INFO:tensorflow:./saved_model_proposed.bin is not in all_model_checkpoint_paths. Manually adding it.
4, 7.20969371108, 0.399106677051
0.352916314455, 0.194526269849, 0.209488910035
INFO:tensorflow:./saved_model_proposed.bin is not in all_model_checkpoint_paths. Manually adding it.
5, 7.14933534571, 0.336986821514
0.354184277261, 0.195177383708, 0.209841314
INFO:tensorflow:./saved_model_proposed.bin is not in all_model_checkpoint_paths. Manually adding it.
6, 7.10010188764, 0.28

0.416596224289, 0.239240330174, 0.251930299757
51, 5.25753857638, 0.199577530494
0.41842772612, 0.24043008291, 0.252394136558
INFO:tensorflow:Restoring parameters from ./saved_model_proposed.bin
0.437447168216, 0.237404500674, 0.258798735565


In [25]:
config = tf.ConfigProto()
config.gpu_options.allow_growth = True
sess = tf.Session(config=config)
sess.run(tf.global_variables_initializer())

train_loss = 0.
train_auc = 0.
train_hit = 0.
loss_cnt = 0.
best_ndcg = 0.
counter = 0
base = True  # whether to train the base model alone 
saver = tf.train.Saver(max_to_keep=None)


# sess.graph.finalize()
for epoch in range(100):
    train_loss = 0.
    train_kl = 0.
    loss_cnt = 0.

    for i in range(0, user_biz_train.shape[0], batch_size):
        uy_batch = user_biz_train[i:i+batch_size, :]
        ux_batch = user_non_zero_ind[i:i+batch_size]
        if base:
            _, loss_val, kl_val = sess.run([opt_base, loss, kl_div], feed_dict={ux: ux_batch, uy: uy_batch})
        else:
            _, loss_val, kl_val = sess.run([opt, loss, kl_div], feed_dict={ux: ux_batch, uy: uy_batch, loss_beta: max(0.9 ** epoch, 0.1)})

        train_loss += loss_val
        train_kl += kl_val
        loss_cnt += 1

    print("{}, {}, {}".format(epoch, train_loss/loss_cnt, train_kl/loss_cnt))

    val_hit = []
    val_recall = []
    val_ndcg = []
    for i in range(0, len(val_set_u), batch_size):
        u_batch = val_set_u[i:i+batch_size]
        mask_batch = val_set_mask[i:i+batch_size]
        set_batch = val_set_set[i:i+batch_size]
        y_val = sess.run(y_inference, feed_dict={ux: u_batch})
        y_val *= np.array(mask_batch)
        hit_, recall_, ndcg_ = evaluate(u_batch, mask_batch, set_batch, y_val, cutoff=[5, 20, 100])
        val_hit += hit_
        val_recall += recall_
        val_ndcg += ndcg_
    val_hit = np.array(val_hit)
    val_recall = np.array(val_recall)
    val_ndcg = np.array(val_ndcg)
    print("{}, {}, {}".format(val_hit.mean(), val_recall.mean(), val_ndcg.mean()))
    
    if val_ndcg.mean() >= best_ndcg:
        best_ndcg = val_ndcg.mean()
        counter = 0
        saver.save(sess, './saved_model_%s.bin'%('base' if base else 'proposed'))

    counter += 1
    if counter > 3:
        break

saver.restore(sess, './saved_model_%s.bin'%('base' if base else 'proposed'))
test_hit = []
test_recall = []
test_ndcg = []
for i in range(0, len(test_set_u), batch_size):
    u_batch = test_set_u[i:i+batch_size]
    mask_batch = test_set_mask[i:i+batch_size]
    set_batch = test_set_set[i:i+batch_size]
    y_val = sess.run(y_inference, feed_dict={ux: u_batch})
    y_val *= np.array(mask_batch)
    hit_, recall_, ndcg_ = evaluate(u_batch, mask_batch, set_batch, y_val, cutoff=[5, 20, 100])
    test_hit += hit_
    test_recall += recall_
    test_ndcg += ndcg_
test_hit = np.array(test_hit)
test_recall = np.array(test_recall)
test_ndcg = np.array(test_ndcg)
print("{}, {}, {}".format(test_hit.mean(), test_recall.mean(), test_ndcg.mean()))

0, 9.06072061126, 1.44003292247
0.20583262891, 0.116154571115, 0.147707240746
INFO:tensorflow:./saved_model_base.bin is not in all_model_checkpoint_paths. Manually adding it.
1, 7.67372899013, 0.978538833223
0.348267117498, 0.195468570116, 0.207826062979
INFO:tensorflow:./saved_model_base.bin is not in all_model_checkpoint_paths. Manually adding it.
2, 7.34630424053, 1.04444442622
0.353761622992, 0.195245293881, 0.207425906501
3, 7.22505988301, 1.09021588005
0.35235277543, 0.19451892674, 0.208453443263
INFO:tensorflow:./saved_model_base.bin is not in all_model_checkpoint_paths. Manually adding it.
4, 7.14264832316, 1.11231974761
0.352634544942, 0.194537610614, 0.208701231752
INFO:tensorflow:./saved_model_base.bin is not in all_model_checkpoint_paths. Manually adding it.
5, 7.07674529531, 1.12720642487
0.352211890673, 0.194662059005, 0.209462720983
INFO:tensorflow:./saved_model_base.bin is not in all_model_checkpoint_paths. Manually adding it.
6, 7.02142192222, 1.14556811146
0.352071005